1.计算给定数据集的香农熵


In [2]:
import pickle
import operator
from math import log
def calcShannonEnt(dataSet):
    # 计算数据集中实例的个数
    numEntries = len(dataSet)
    # 所有可能分类创建字典
    labelCount={}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCount.keys():
            # 每个键值都记录了当前类别出现的次数
            labelCount[currentLabel] =0
            labelCount[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCount:
        prob = float (labelCount[key])/numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

# 按照给定特征值划分数据集
#  dataSet待划分的数据集  axis 划分数据集的特征，value 需要返回的特征的值
def splitDataSet(dataSet,axis,value):
    # 创建返回的数据集列表
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            # 抽取axis
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet


# 选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet):
    # 特征数量
    numFeatures = len(dataSet[0]) - 1
    #计算数据集的香农熵
    baseEntropy = calcShannonEnt(dataSet)
    #信息增益
    bestInfoGain = 0.0
    # 最优特征值的索引值
    bestFeature = -1
    # 遍历所有特征
    for i in range(numFeatures):
          #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        # set集合每个特征值都不相同
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


# 统计classList中出现次数最多的类标签
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
        # 根据字典的值降序排序
    sortedClassCount = sorted(classCount.items(),key = operator.itemgetter(1),reverse = True)
    # 返回出现次数最多的分类名称
    return sortedClassCount[0][0]


# 创建树的代码
# dataSet 数据集
# labels 标签列表
def createTree(dataSet,labels):
    # 取分类标签
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 遍历完所有特征时返回出现次数最多的类别
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 选择最优特征值
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    # 删除已经使用的特征标签
    del(labels[bestFeat])
    #得到训练集中所有最优特征的属性值
    featValues =[example[bestFeat] for example in dataSet]
    # 去掉重复的属性值
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subDataSet = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subDataSet)
    return myTree


# 使用决策树的分类函数
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__== 'dict':
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
                
    return classLabel

# 使用pickle模块存储决策树
# def storeTree(inputTree, filename):
#     with open(filename, 'wb') as fw:
#         pickle.dump(inputTree, fw)
 
# def createDataSet():
#     dataSet = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
#     labels =['no surfacing','flippers']
#     return dataSet,labels


if __name__ =="__main__":
    fr = open('lenses.txt')
    lenses = [inst.strip().split('\t') for inst in fr.readlines()]
    lensesLabels = ['age','prescript','astigmatic','tearRate']
    lensesTree = createTree(lenses,lensesLabels)
    print(lensesTree)





{'tearRate': {'hard': 'hard', 'soft': 'soft', 'no lenses': 'no lenses'}}
